In [1]:
import bw2data as bd
import bw2io as bi
# from premise_gwp import add_premise_gwp  # only add if you want to use the premise

def base_setup_with_ecoinvent(
    project_name: str,
    db_name: str,
    db_loc: str,
    reparametrize_lognormals: bool = False,
    # setup_premise_gwp: bool = True, # only add if you want to use the premise
):
    """
    This method sets up a Brightway project with the ecoinvent database.

    Parameters
    ----------
    db_name : str
        The name of the database to import
    db_loc : str
        The location of the database to import (e.g. "C:/Users/.../ecoinvent 3.7.1_cutoff_ecoSpold02/datasets"))
    reparametrize_lognormals : bool, optional
        Whether or not to reparametrize lognormal distributions, by default True (recommended)
        This sets the mean of the lognormal distribution to static value of the LCA calculation.
        This is recommended because the default behavior of Brightway is to set the mean of the lognormal distribution to the value of the LCA calculation.
        This can lead to problems when performing Monte Carlo LCA calculations, because the mean of the lognormal distribution will change with each iteration.
        This can lead to a large number of iterations being required to reach convergence.
    setup_premise_gwp : bool, optional
        Whether or not to setup the premise_gwp method, by default True
        This method provides characterization factors for biogenic CO2 flows and hydrogen emissions to air.
    """

    # set the project, it will create a new project if it does not exist
    bd.projects.set_current(project_name)

    # imports biosphere flows, it will skip if already done
    bi.bw2setup()

    # get the path to the datasets folder
    db_dir = db_loc
    # give a name to the database
    data_base_name = db_name
    # check if the database is already in the project, and skip if yes
    if data_base_name in bd.databases:
        print("Database has already been imported")

    else:
        # import database
        db = bi.SingleOutputEcospold2Importer(
            db_dir,
            data_base_name,
            reparametrize_lognormals=reparametrize_lognormals,
        )

        db.apply_strategies()

        db.statistics()

        # db.drop_unlinked(True)
        # writes database into sql
        db.write_database()
        # Because some of the scenarios can yield LCI databases
        # containing net negative emission technologies (NET),
        # it is advised to account for biogenic CO2 flows when calculating
        # Global Warming potential indicators.
        # `premise_gwp` provides characterization factors for such flows.
        # It also provides factors for hydrogen emissions to air.
        # if setup_premise_gwp: # only add if you want to use the premise
        #     add_premise_gwp()

In [2]:
def get_method(method: str, include_premise_recommended: bool = False):
    """
    Give a name of an LCIA method and it will return the mid points of that method

    Parameters
    ----------
    method : str
        The name of the LCIA method
        for example, EN15804, ReCiPe 2016 v1.03, EF v3.1
    include_premise_recommended : bool
        If True, it will include the recommended methods from premise
    """
    methods = [m for m in bd.methods if method == m[0]]
    if include_premise_recommended:
        if (
            not ("IPCC 2021", "climate change", "GWP 100a, incl. H and bio CO2")
            in methods
            and ("IPCC 2021", "climate change", "GWP 100a, incl. H and bio CO2")
            in bd.methods
        ):
            methods.append(
                ("IPCC 2021", "climate change", "GWP 100a, incl. H and bio CO2")
            )
        if (
            not ("IPCC 2021", "climate change", "GWP 20a, incl. H and bio CO2")
            in methods
            and ("IPCC 2021", "climate change", "GWP 20a, incl. H and bio CO2")
            in bd.methods
        ):
            methods.append(
                ("IPCC 2021", "climate change", "GWP 20a, incl. H and bio CO2")
            )
        if (
            not ("IPCC 2021", "climate change", "GWP 100a, incl. H") in methods
            and ("IPCC 2021", "climate change", "GWP 100a, incl. H") in bd.methods
        ):
            methods.append(("IPCC 2021", "climate change", "GWP 100a, incl. H"))
        if (
            not ("IPCC 2021", "climate change", "GWP 20a, incl. H") in methods
            and ("IPCC 2021", "climate change", "GWP 20a, incl. H") in bd.methods
        ):
            methods.append(("IPCC 2021", "climate change", "GWP 20a, incl. H"))
    return methods

In [3]:
import bw2calc as bc
from tqdm import tqdm

def staticLCA(
    demands: list[dict[bd.Node, float]], methods: list[tuple]
) -> dict[int : dict[tuple, list[float]]]:
    """
    This method performs static LCA calculations for a list of demands and a list of methods.
    
    If you plan to run LCAs for activities form different databases, split the demands into separate lists.
    Each list of demands should contain demands for activities from the same database. that way the lci is only performed once for each database.
    And it runs much faster.

    Parameters
    ----------
    demands : list
        A list of demands, where each demand is a dictionary of the form:
        {
            activity name: amount
        }
        methods : list
        A list of methods, where each method is a tuple of the form:
        ("foo", "bar", "baz")
    """
    if len(methods) == 0:
        raise ValueError(
            "No methods were specified. Please specify at least one method to perform LCA calculations"
        )
    # assert that methods are lists of tuples
    assert (
        isinstance(methods, list)
        and isinstance(methods[0], tuple)
        and isinstance(methods[0][0], str)
    ), "Methods must be a list of tuples of strings"

    # Create all demands with a comprehension
    all_demands = {k: 1 for demand in demands for k in demand}
    # Initialize LCA object
    lca = bc.LCA(demand=all_demands, method=methods[0])
    lca.lci()
    # Create a list of characterization matrices
    C_matrices = {}

    for method in methods:
        lca.switch_method(method)
        C_matrices[method] = lca.characterization_matrix.copy()

    # Create container for results
    results = {}

    # Adding a progress bar for iterating through demands
    pbar = tqdm(total=len(demands), desc="Initializing LCA Calculations")
    for demand in demands:
        first_key = next(iter(demand))
        key_tuple = (
            first_key["name"],
            first_key["location"],
            first_key["code"],
            first_key["database"],
        )

        # Updating the progress bar description
        pbar.set_description(
            f"Performing LCA for activities of database: {key_tuple[3]}"
        )

        results[key_tuple] = {method: None for method in methods}

        # Convert to integer ids
        int_demand = {key.id: value for key, value in demand.items()}
        lca.lci(int_demand)
        for method in methods:
            lca.switch_method(method)
            results[key_tuple][method] = (C_matrices[method] * lca.inventory).sum()

        pbar.update(1)  # Update the progress bar after each demand is processed

    pbar.close()  # Close the progress bar after the loop

    return results

In [4]:
import numpy as np

def monteCarloLCA(
    demands: list[dict[bd.Node, float]],
    methods: list[tuple[str, str, str]],
    iterations: int = 20,
) -> dict[int : dict[tuple, np.ndarray]]:
    """
    This method performs Monte Carlo LCA calculations for a list of demands and a list of methods.

    Parameters
    ----------
    demands : list
        A list of demands, where each demand is a dictionary of the form:
        {
            activity name: amount
        }
    methods : list
        A list of methods, where each method is a tuple of the form:
        ("foo", "bar", "baz")
    iterations : int, optional
        The number of Monte Carlo iterations to perform, by default 20"""

    if len(methods) == 0:
        raise ValueError(
            "The methods list is empty, make sure you set the project in brightway, before calling the methods"
        )
    assert (
        isinstance(methods, list)
        and isinstance(methods[0], tuple)
        and isinstance(methods[0][0], str)
    ), "Methods must be a list of tuples of strings"
    # Create all demands with a comprehension
    all_demands = {k: 1 for demand in demands for k in demand}

    # Initialize LCA object
    lca = bc.LCA(demand=all_demands, method=methods[0], use_distributions=True)
    lca.lci()

    # Create characterization matrices using a loop
    C_matrices = {}
    for method in methods:
        lca.switch_method(method)
        C_matrices[method] = lca.characterization_matrix.copy()

    # Initialize container for results
    results = {}
    for demand in demands:
        first_key = next(iter(demand))
        key_tuple = (
            first_key["name"],
            first_key["location"],
            first_key["code"],
            first_key["database"],
        )
        results[key_tuple] = {method: [] for method in methods}

    # Adding a progress bar for the Monte Carlo iterations
    with tqdm(total=iterations, desc="Monte Carlo LCA Progress") as pbar:
        for iteration in range(1, iterations + 1):
            # Update progress bar with current iteration and demand
            pbar.set_description(
                f"Monte Carlo Iterations for activities of database: {key_tuple[3]}"
            )

            # Resample all matrices
            next(lca)
            for demand in demands:
                first_key = next(iter(demand))
                key_tuple = (
                    first_key["name"],
                    first_key["location"],
                    first_key["code"],
                    first_key["database"],
                )
                # Convert to integer ids
                int_demand = {key.id: value for key, value in demand.items()}
                lca.lci(int_demand)
                for method in methods:
                    lca.switch_method(method)
                    results[key_tuple][method].append(
                        (C_matrices[method] * lca.inventory).sum()
                    )

            pbar.update(1)  # Update the progress bar after each iteration

    # Convert lists to numpy arrays
    for key_tuple, methods_data in results.items():
        for method, values in methods_data.items():
            results[key_tuple][method] = np.array(values)

    return results

Use Examples:

In [5]:
bd.projects.set_current('Bhorizionv2')
db = bd.Database('ecoinvent391cutoff')
methods = get_method('EF v3.1')

In [6]:
act1 = {db.random(): 1}
act2 = {db.random(): 2}
act3 = {db.random(): 3}
activities = [act1, act2, act3]

In [7]:
static_results = staticLCA(activities, methods)
static_results

Performing LCA for activities of database: ecoinvent391cutoff: 100%|██████████| 3/3 [00:00<00:00, 12.04it/s]


{('clinker production',
  'RoW',
  '81dff46f8f38958d90748c7947cd8934',
  'ecoinvent391cutoff'): {('EF v3.1',
   'acidification',
   'accumulated exceedance (AE)'): 0.0,
  ('EF v3.1', 'climate change', 'global warming potential (GWP100)'): 0.0,
  ('EF v3.1',
   'climate change: biogenic',
   'global warming potential (GWP100)'): 0.0,
  ('EF v3.1',
   'climate change: fossil',
   'global warming potential (GWP100)'): 0.0,
  ('EF v3.1',
   'climate change: land use and land use change',
   'global warming potential (GWP100)'): 0.0,
  ('EF v3.1',
   'ecotoxicity: freshwater',
   'comparative toxic unit for ecosystems (CTUe)'): 0.0,
  ('EF v3.1',
   'ecotoxicity: freshwater, inorganics',
   'comparative toxic unit for ecosystems (CTUe)'): 0.0,
  ('EF v3.1',
   'ecotoxicity: freshwater, organics',
   'comparative toxic unit for ecosystems (CTUe)'): 0.0,
  ('EF v3.1',
   'energy resources: non-renewable',
   'abiotic depletion potential (ADP): fossil fuels'): 0.0,
  ('EF v3.1',
   'eutrophica

In [8]:
monte_carlo_results = monteCarloLCA(activities, methods, iterations=10)
monte_carlo_results

Monte Carlo Iterations for activities of database: ecoinvent391cutoff: 100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


{('clinker production',
  'RoW',
  '81dff46f8f38958d90748c7947cd8934',
  'ecoinvent391cutoff'): {('EF v3.1',
   'acidification',
   'accumulated exceedance (AE)'): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  ('EF v3.1',
   'climate change',
   'global warming potential (GWP100)'): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  ('EF v3.1',
   'climate change: biogenic',
   'global warming potential (GWP100)'): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  ('EF v3.1',
   'climate change: fossil',
   'global warming potential (GWP100)'): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  ('EF v3.1',
   'climate change: land use and land use change',
   'global warming potential (GWP100)'): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  ('EF v3.1',
   'ecotoxicity: freshwater',
   'comparative toxic unit for ecosystems (CTUe)'): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  ('EF v3.1',
   'ecotoxicity: freshwater, inorganics',
   'comparative toxic unit for ecosyst